In [1]:
import cv2

from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np

In [2]:
fd = cv2.CascadeClassifier(cv2.data.haarcascades+"haarcascade_frontalface_alt2.xml")
model = load_model(r'C:\Users\KIIT\Documents\MachineLearning_scripts\Trained Models\maskdetect_mobilenet_v2_predictor.model')

In [3]:
video_capture = cv2.VideoCapture(0)
while True:
    ret, frame = video_capture.read()
    gr = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    f = fd.detectMultiScale(gr,scaleFactor=1.1,minNeighbors=5,minSize=(60, 60),flags=cv2.CASCADE_SCALE_IMAGE)
    f_list=[]
    preds=[]
    
    for (x, y, w, h) in f:
        f_frame = frame[y:y+h,x:x+w]
        f_frame = cv2.cvtColor(f_frame, cv2.COLOR_BGR2RGB)
        f_frame = cv2.resize(f_frame, (224, 224))
        f_frame = img_to_array(f_frame)
        f_frame = np.expand_dims(f_frame, axis=0)
        f_frame =  preprocess_input(f_frame)
        f_list.append(f_frame)
        if len(f_list)>0:
            preds = model.predict(f_list)
        for pred in preds:
            (mask, withoutMask) = pred
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        cv2.putText(frame, label, (x, y- 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (x, y), (x + w, y + h),color, 2)

    cv2.imshow('Video',frame)
    k = cv2.waitKey(1)   
    if(k == ord('q')):
        break       
video_capture.release()
cv2.destroyAllWindows()